![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [6]:
#region imports
from AlgorithmImports import *
#endregion
import pandas as pd
from footprint_utils import classify_aggressor # 导入需要调试的函数
from datetime import datetime, timedelta

# --- 1. 初始化和历史数据请求 ---
qb = QuantBook()
#symbol = qb.add_future(Futures.Metals.GOLD, Resolution.SECOND).symbol
symbol = qb.add_future(Futures.Indices.NASDAQ_100_E_MINI, Resolution.SECOND).symbol

history_date = datetime(2024, 11, 15)

df_history = qb.history(symbol, history_date, history_date + timedelta(days=1),
                        resolution=Resolution.SECOND,
                        extended_market_hours=True,
                        data_mapping_mode=DataMappingMode.OPEN_INTEREST_ANNUAL, # 数据映射模式，这个会根据交易量切换到当年后续更大的合约，Warning, 但正确性有待验证
                        data_normalization_mode=DataNormalizationMode.BACKWARDS_RATIO, # 数据连续模式，ATAS是RAW, tradingview 是BACKWARDS_RATIO，能够使得连续。注意，实盘需要使用当期合约数据
                        fill_forward=True
                        )
print(df_history[df_history['volume']>=100])
if df_history.empty:
    qb.Debug("No history data found for the selected date or symbol.")
else:
    # --- 2. 准备数据进行调试 ---
    df_debug = df_history.copy().reset_index()
    df_debug = df_debug.rename(columns={
        'close': 'trade_price',
        'bidclose': 'bid_price',
        'askclose': 'ask_price'
    })
    
    # 筛选出有成交量且报价完整的行
    df_debug = df_debug[['trade_price', 'bid_price', 'ask_price', 'volume']].dropna()
    df_debug = df_debug[df_debug['volume'] > 0].head(100) # 只看前20条有成交的数据
    
    qb.Debug(f"--- Aggressor Classification Debug (for first {len(df_debug)} trades) ---")

    class SimpleTradeBar:
        def __init__(self, price):
            self.close = price
    class SimpleQuoteBar:
        def __init__(self, bid, ask):
            self.bid = type('Bar', (), {'close': bid})()
            self.ask = type('Bar', (), {'close': ask})()
            
    aggressor_results = []
    
    # --- 3. 逐行打印分类逻辑 ---
    for index, row in df_debug.iterrows():
        trade_price = row['trade_price']
        bid_price = row['bid_price']
        ask_price = row['ask_price']
        
        # 模拟对象
        trade_bar = SimpleTradeBar(trade_price)
        quote_bar = SimpleQuoteBar(bid_price, ask_price)
        
        # 进行分类
        aggressor = classify_aggressor(trade_bar, quote_bar)
        aggressor_results.append(aggressor)
        
        # 打印判断过程
        print(f"Trade at {trade_price:8.2f}: Bid={bid_price:8.2f}, Ask={ask_price:8.2f}  => Classified as: {aggressor.upper()}")

    # --- 4. 统计总体分类结果 ---
    s_results = pd.Series(aggressor_results)
    print("\n--- Overall Classification Stats ---")
    print(s_results.value_counts())
    

                                           askclose       askhigh  \
expiry     symbol time                                              
1899-12-30 /NQ    2024-11-15 01:00:08  21838.210146  21841.867919   
                  2024-11-15 01:07:53  21818.614934  21828.020636   
                  2024-11-15 01:12:25  21805.028921  21809.993041   
                  2024-11-15 01:21:54  21801.371148  21804.506382   
                  2024-11-15 03:09:02  21770.280079  21772.631505   
...                                             ...           ...   
                  2024-11-15 16:00:14  21430.107206  21430.629745   
                  2024-11-15 16:00:21  21430.107206  21430.629745   
                  2024-11-15 16:00:28  21431.936092  21432.981170   
                  2024-11-15 16:00:48  21440.819255  21441.080524   
                  2024-11-15 16:00:50  21441.341794  21443.170680   

                                             asklow       askopen  asksize  \
expiry     symbol time  